### This is just to get the ratios in, so we can get a list of gene names:

In [ ]:
%run -i read_counts.py

In [19]:
cd BLASTING

/home/dreiss/scratch/halo/DVU_for_discovery_prop/discovery2014_local/BLASTING


### some ideas from cmonkey_experimental.R

In [ ]:
! wget 'http://rsat.ulb.ac.be/rsat/data/genomes/Desulfovibrio_desulfuricans_G20/genome/Desulfovibrio_desulfuricans_G20_aa.fasta'

In [ ]:
!wget 'http://rsat.ulb.ac.be/rsat/data/genomes/Desulfovibrio_vulgaris_Hildenborough/genome/Desulfovibrio_vulgaris_Hildenborough_aa.fasta'

In [ ]:
!./formatdb -p T -o F -i Desulfovibrio_desulfuricans_G20_aa.fasta -l temp1.log
!./formatdb -p T -o F -i Desulfovibrio_vulgaris_Hildenborough_aa.fasta -l temp2.log
!./blastall -d Desulfovibrio_vulgaris_Hildenborough_aa.fasta -i Desulfovibrio_desulfuricans_G20_aa.fasta -p blastp -F "m S" -m 8 -b 200000 >blasted_out.tab

In [20]:
%%R
blast.out <- readLines('blasted_out.tab')
out <- t( sapply( strsplit( blast.out, "\t" ), cbind ) )
out <- data.frame( `Query id`=out[ ,1 ], `Subject id`=out[ ,2 ], `% identity`=as.numeric( out[ ,3 ] ),
                    `alignment length`=as.integer( out[ ,4 ] ), mismatches=as.integer( out[ ,5 ] ),
                    `gap openings`=as.integer( out[ ,6 ] ), `q. start`=as.integer( out[ ,7 ] ),
                    `q. end`=as.integer( out[ ,8 ] ), `s. start`=as.integer( out[ ,9 ] ),
                    `s. end`=as.integer( out[ ,10 ] ), `e-value`=as.numeric( out[ ,11 ] ),
                    `bit score`=as.numeric( out[ ,12 ] ) )

e.cutoff <- 1e-5
out <- subset( out, e.value <= e.cutoff )

In [21]:
%%R
read.fasta <- function( fname ) {
    lines <- readLines( gzfile( fname ) )
    lines <- lines[ lines != "" ]
    starts <- grep( "^>", lines, perl=T )
    stops <- c( starts[ 2:length( starts ) ], length( lines ) + 1 )
    seqs <- sapply( 1:length( starts ), function( i ) paste( lines[ ( starts[ i ] + 1 ):( stops[ i ] - 1 ) ],
                                                            collapse="", sep="" ) )
    names( seqs ) <- gsub( "^>", "", lines[ starts ], perl=T )
    seqs
}

fraction.cutoff <- 0.5
lens1 <- nchar( read.fasta( 'Desulfovibrio_vulgaris_Hildenborough_aa.fasta' ) )
lens2 <- nchar( read.fasta( 'Desulfovibrio_desulfuricans_G20_aa.fasta' ) )
out <- cbind( out, fraction.x=out$alignment.length / lens1[ as.character( out$Subject.id ) ],
               fraction.y=out$alignment.length / lens2[ as.character( out$Query.id ) ] )
out <- subset( out, fraction.x >= fraction.cutoff & fraction.y >= fraction.cutoff )

In [22]:
print all_freqs.keys()
x1 = all_freqs['Desulfovibrio_vulgaris_Hildenborough_uid57645'].copy()
x2 = all_freqs['Desulfovibrio_alaskensis_G20_uid57941'].copy()
sizeme(x2.head(),60,6)
genes1 = x1.index.values
%Rpush genes1
genes2 = x2.index.values
%Rpush genes2
#%R save.image('qqq')

['Desulfovibrio_alaskensis_G20_uid57941', 'Desulfovibrio_vulgaris_Hildenborough_uid57645', 'Methanococcus_maripaludis_S2_uid58035', 'Methanospirillum_hungatei_JF_1_uid58181']


In [ ]:
!wget 'http://rsat.ulb.ac.be/rsat/data/genomes/Desulfovibrio_vulgaris_Hildenborough/genome/feature_names.tab'
!mv -v feature_names.tab Desulfovibrio_vulgaris_Hildenborough_feature_names.tab

!wget 'http://rsat.ulb.ac.be/rsat/data/genomes/Desulfovibrio_desulfuricans_G20/genome/feature_names.tab'
!mv -v feature_names.tab Desulfovibrio_desulfuricans_G20_feature_names.tab

In [27]:
%%R
fname <- 'Desulfovibrio_vulgaris_Hildenborough_feature_names.tab'
nskip <- sum( substr( readLines( gzfile( fname ), n=20 ), 1, 2 ) == "--" )
feature.names1 <- read.delim( gzfile( fname ), head=F, as.is=T, skip=nskip, row.names=NULL, comment='' )
colnames( feature.names1 ) <- c( "id", "names", "type" )

fname <- 'Desulfovibrio_desulfuricans_G20_feature_names.tab'
nskip <- sum( substr( readLines( gzfile( fname ), n=20 ), 1, 2 ) == "--" )
feature.names2 <- read.delim( gzfile( fname ), head=F, as.is=T, skip=nskip, row.names=NULL, comment='' )
colnames( feature.names2 ) <- c( "id", "names", "type" )

genes1 <- unlist(genes1)
genes2 <- unlist(genes2)

## Note - Query.id comes from env2 species; Subject.id comes from env1 species
names1 <- unique( subset( feature.names1, names %in% genes1 )[ , c( "id", "names" ) ] )
names2 <- unique( subset( feature.names2, names %in% genes2 )[ , c( "id", "names" ) ] )
out <- merge( out, names2, by.x='Query.id', by.y='id', all.x=T, all.y=F )
out <- merge( out, names1, by.x='Subject.id', by.y='id', all.x=T, all.y=F )
  
## Sort it so first one (which is kept) is the best
out <- out[ order( -out$bit.score, out$e.value ), ]
## Look for reciprocal best hits
##out <- out[ ! duplicated( out[ ,1:2 ] ), ] ## First get rid of dupes (keeping best hit b/c it is sorted)
out <- subset( out, ! duplicated( Subject.id ) & ! duplicated( Query.id ) )
out <- subset( out, ! is.na( names.y ) & ! is.na( names.x ) )
#print(head(out))
blast_out <- out; rm(out)
save.image('blasting.RData')
NULL

NULL


In [28]:
%Rpull blast_out
sizeme(blast_out.head(),60)

,Subject.id,Query.id,X..identity,alignment.length,mismatches,gap.openings,q..start,q..end,s..start,s..end,e.value,bit.score,fraction.x,fraction.y,names.x,names.y,names.x.1,names.y.1,names.x.2,names.y.2
0,2765,2875,85.36,1380,201,1,1,1380,1,1379,0,2420,0.996390,0.995671,Dde_2998,DVU2929,Dde_2998,DVU2929,Dde_2998,DVU2929
1,2764,2874,83.70,1368,222,1,1,1367,1,1368,0,2385,0.997085,0.994909,Dde_2997,DVU2928,Dde_2997,DVU2928,Dde_2997,DVU2928
2,2854,3106,82.68,1218,207,3,1,1217,1,1215,0,2106,1.002469,1.000822,Dde_3237,DVU3025,Dde_3237,DVU3025,Dde_3237,DVU3025
3,1803,2009,79.81,1233,249,0,1,1233,1,1233,0,2079,0.999190,0.995961,Dde_2081,DVU1834,Dde_2081,DVU1834,Dde_2081,DVU1834
4,1802,2008,79.82,1194,238,2,1,1194,1,1191,0,1985,1.002519,1.000000,Dde_2080,DVU1833,Dde_2080,DVU1833,Dde_2080,DVU1833
